In [1]:
from Bio import SeqIO
import pandas as pd
from BCBio import GFF


In [2]:
%cd "/home/andrei/Data/HeteroR/test_dir/GRF"
rgi = pd.read_csv("DA62886_rgi_table.tsv", sep="\t")

/home/andrei/Data/HeteroR/test_dir/GRF


In [3]:
rgi.columns

Index(['ORF_ID', 'Contig', 'Start', 'Stop', 'Orientation', 'Cut_Off',
       'Pass_Bitscore', 'Best_Hit_Bitscore', 'Best_Hit_ARO', 'Best_Identities',
       'ARO', 'Model_type', 'SNPs_in_Best_Hit_ARO', 'Other_SNPs', 'Drug Class',
       'Resistance Mechanism', 'AMR Gene Family', 'Predicted_DNA',
       'Predicted_Protein', 'CARD_Protein_Sequence',
       'Percentage Length of Reference Sequence', 'ID', 'Model_ID', 'Nudged',
       'Note'],
      dtype='object')

In [4]:
rgi["Cut_Off"].unique()

array(['Loose', 'Strict', 'Perfect'], dtype=object)

Filter out "Loose" hits, leave only "Strict" and "Perfect"

In [5]:
pd.set_option('display.max_columns', None)
rgi_notLoose = rgi[rgi["Cut_Off"] != "Loose"]
rgi_notLoose

,ORF_ID,Contig,Start,Stop,Orientation,Cut_Off,Pass_Bitscore,Best_Hit_Bitscore,Best_Hit_ARO,Best_Identities,ARO,Model_type,SNPs_in_Best_Hit_ARO,Other_SNPs,Drug Class,Resistance Mechanism,AMR Gene Family,Predicted_DNA,Predicted_Protein,CARD_Protein_Sequence,Percentage Length of Reference Sequence,ID,Model_ID,Nudged,Note
93,HOKEGFAO_00108 Multidrug transporter EmrE,NaN,NaN,NaN,NaN,Strict,190,216.083,Escherichia coli emrE,98.18,3004039,protein homolog model,NaN,NaN,macrolide antibiotic,antibiotic efflux,small multidrug resistance (SMR) antibiotic ef...,NaN,MNPYIYLGGAILAEVIGTTLMKFSEGFTRLWPSVGTIICYCASFWL...,MNPYIYLGGAILAEVIGTTLMKFSEGFTRLWPSVGTIICYCASFWL...,100.00,gnl|BL_ORD_ID|2380|hsp_num:0,2656,NaN,NaN
301,HOKEGFAO_00337 UDP-glucose 6-dehydrogenase,NaN,NaN,NaN,NaN,Strict,700,759.599,ugd,94.59,3003577,protein homolog model,NaN,NaN,peptide antibiotic,antibiotic target alteration,pmr phosphoethanolamine transferase,NaN,MKITISGTGYVGLSNGLLISQNHNVVALDILPSRVAMLNDRISPIV...,MKITISGTGYVGLSNGLLIAQNHEVVALDILPSRVAMLNDRISPIV...,100.00,gnl|BL_ORD_ID|916|hsp_num:0,842,NaN,NaN
347,HOKEGFAO_00388 Multidrug resistance protein MdtA,NaN,NaN,NaN,NaN,Strict,725,826.239,mdtA,98.80,3000792,protein homolog model,NaN,NaN,aminocoumarin antibiotic,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,NaN,MKGSYKSRWVIVIVVVIAAIAAFWFWQGRNDSQSAAPGATKQAQQS...,MKGSYKSRWVIVIVVVIAAIAAFWFWQGRNDSRSAAPGATKQAQQS...,100.00,gnl|BL_ORD_ID|360|hsp_num:0,387,NaN,NaN
348,HOKEGFAO_00389 Multidrug resistance protein MdtB,NaN,NaN,NaN,NaN,Strict,1800,2076.600,mdtB,99.71,3000793,protein homolog model,NaN,NaN,aminocoumarin antibiotic,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,NaN,MQVLPPSSTGGPSRLFIMRPVATTLLMVAILLAGIIGYRALPVSAL...,MQVLPPSSTGGPSRLFIMRPVATTLLMVAILLAGIIGYRALPVSAL...,100.00,gnl|BL_ORD_ID|776|hsp_num:0,820,NaN,NaN
349,HOKEGFAO_00390 Multidrug resistance protein MdtC,NaN,NaN,NaN,NaN,Strict,1800,2061.960,mdtC,99.51,3000794,protein homolog model,NaN,NaN,aminocoumarin antibiotic,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,NaN,MKFFALFIYRPVATILLSVAITLCGILGFRMLPVAPLPQVDFPVIM...,MKFFALFIYRPVATILLSVAITLCGILGFRMLPVAPLPQVDFPVII...,100.00,gnl|BL_ORD_ID|1241|hsp_num:0,1315,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,HOKEGFAO_04953 Dihydropteroate synthase,NaN,NaN,NaN,NaN,Perfect,500,528.865,sul2,100.00,3000412,protein homolog model,NaN,NaN,sulfonamide antibiotic,antibiotic target replacement,sulfonamide resistant sul,NaN,MNKSLIIFGIVNITSDSFSDGGRYLAPDAAIAQARKLMAEGADVID...,MNKSLIIFGIVNITSDSFSDGGRYLAPDAAIAQARKLMAEGADVID...,100.00,gnl|BL_ORD_ID|894|hsp_num:0,952,NaN,NaN
4425,HOKEGFAO_04954 hypothetical protein,NaN,NaN,NaN,NaN,Strict,500,355.140,APH(3'')-Ib,99.43,3002639,protein homolog model,NaN,NaN,aminoglycoside antibiotic,antibiotic inactivation,APH(3''),NaN,MNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...,MNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...,67.42,gnl|BL_ORD_ID|1729|hsp_num:0,1830,True,loose hit with at least 95 percent identity pu...
4426,HOKEGFAO_04955 Dihydrofolate reductase type 1,NaN,NaN,NaN,NaN,Strict,300,327.791,dfrA14,98.73,3002859,protein homolog model,NaN,NaN,diaminopyrimidine antibiotic,antibiotic target replacement,trimethoprim resistant dihydrofolate reductase...,NaN,MRTLKVSLIAAKAKNGVIGCGPDIPWSAKGEQLLFKALTYNQWLLV...,MKVSLMAAKAKNGVIGCGPDIPWSAKGEQLLFKALTYNQWLLVGRK...,101.91,gnl|BL_ORD_ID|1496|hsp_num:0,1592,NaN,NaN
4427,HOKEGFAO_04956 Aminoglycoside 3'-phosphotransf...,NaN,NaN,NaN,NaN,Strict,500,166.007,APH(3'')-Ib,100.00,3002639,protein homolog model,NaN,NaN,aminoglycoside antibiotic,antibiotic inactivation,APH(3''),NaN,MPNFMVDPKTLQCTGLIDLGRLGTADRYADLALMIANAEENWAAPD...,MNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...,30.34,gnl|BL_ORD_ID|1729|hsp_num:0,1830,True,loose hit with at least 95 percent identity pu...


In [6]:
in_file = "DA62886_genomic.gff"

with open(in_file) as f:
    gff = [rec for rec in GFF.parse(f)]

chromosome_genes = [feature for feature in gff[0].features if feature.type == "gene"]  # here we have IDs and positions

Filter the genes with resistance - both in RGI and GBK

naive looping approach

In [7]:
resistance_genes_coords = list()
for orf in rgi_notLoose["ORF_ID"]:
    for gene in chromosome_genes:
        if orf.split(" ")[0] in gene.id:
            resistance_genes_coords.append(gene)


Less naive approach

make sets

In [8]:
rgi_set = set([item.split(" ")[0] for item in rgi_notLoose["ORF_ID"]])

chrom_genes_set = set([item.id[:-5] for item in chromosome_genes])

In [9]:
len(rgi_set)

61

In [10]:
len(chrom_genes_set)

4947

In [11]:
len(chrom_genes_set.intersection(rgi_set))

55

Annotation file lacks some RG? (6)

In [12]:
rg_found = chrom_genes_set.intersection(rgi_set)
rg_notFound = rgi_set.difference(rg_found)
rg_notFound

{'HOKEGFAO_04949',
 'HOKEGFAO_04953',
 'HOKEGFAO_04954',
 'HOKEGFAO_04955',
 'HOKEGFAO_04956',
 'HOKEGFAO_04957'}

What type of table I need:

| RG    | start | stop | span_start | span_stop | strand |
|-------|-------|------|------------|-----------|--------|
| gene1 | 1200  | 3200 | -98800     |  103200   |  1     |
| gene2 | 250000 | 252000 | 150000  | 352000    |  -1    |

- Pay special attention to genes closer than 100kb to the "beginning" of the genome - we need to pull the span from
the end of the genome.

- look for repeats in the entire region: *span_left* + *gene* + *span_right* = *2 x Span + Gene*

Let's make a table with coordinates:

In [13]:
resistance_genes_coords[0:5]


[SeqFeature(FeatureLocation(ExactPosition(90210), ExactPosition(90543), strand=-1), type='gene', id='HOKEGFAO_00108_gene'),
 SeqFeature(FeatureLocation(ExactPosition(354630), ExactPosition(355797), strand=-1), type='gene', id='HOKEGFAO_00337_gene'),
 SeqFeature(FeatureLocation(ExactPosition(414123), ExactPosition(415371), strand=1), type='gene', id='HOKEGFAO_00388_gene'),
 SeqFeature(FeatureLocation(ExactPosition(415370), ExactPosition(418493), strand=1), type='gene', id='HOKEGFAO_00389_gene'),
 SeqFeature(FeatureLocation(ExactPosition(418493), ExactPosition(421571), strand=1), type='gene', id='HOKEGFAO_00390_gene')]

In [14]:
for gene in resistance_genes_coords:
    print(gene.location)

[90210:90543](-)
[354630:355797](-)
[414123:415371](+)
[415370:418493](+)
[418493:421571](+)
[422983:424387](+)
[424383:425106](+)
[560973:562617](-)
[604627:605986](-)
[620511:621480](+)
[730079:731618](-)
[731617:732781](-)
[733196:733811](+)
[733815:737409](+)
[825413:828527](+)
[1009848:1010379](+)
[1010505:1011678](+)
[1011694:1013233](+)
[1019819:1020005](-)
[1520693:1522175](+)
[1549690:1550512](-)
[1750355:1751018](-)
[1751416:1752574](+)
[1752585:1755690](+)
[1807559:1808744](-)
[1823534:1824167](+)
[2004003:2005161](+)
[2005185:2008299](+)
[2008661:2009390](-)
[2009758:2010583](-)
[2479932:2481306](-)
[2564768:2565953](+)
[2726887:2727211](-)
[2727296:2727761](+)
[2755765:2757232](-)
[2757228:2759280](-)
[2759279:2760311](-)
[2789525:2791169](-)
[2997000:2998134](+)
[3266340:3266931](+)
[3307815:3309582](+)
[3676131:3677289](-)
[3762149:3765299](-)
[3765321:3766515](-)
[3766656:3767304](+)
[3967382:3968060](-)
[4110033:4111266](+)
[4189562:4191311](+)
[4425277:4426504](-)
[44

In [15]:
help(resistance_genes_coords[0].location)

Help on FeatureLocation in module Bio.SeqFeature object:

class FeatureLocation(builtins.object)
 |  FeatureLocation(start, end, strand=None, ref=None, ref_db=None)
 |  
 |  Specify the location of a feature along a sequence.
 |  
 |  The FeatureLocation is used for simple continuous features, which can
 |  be described as running from a start position to and end position
 |  (optionally with a strand and reference information).  More complex
 |  locations made up from several non-continuous parts (e.g. a coding
 |  sequence made up of several exons) are described using a SeqFeature
 |  with a CompoundLocation.
 |  
 |  Note that the start and end location numbering follow Python's scheme,
 |  thus a GenBank entry of 123..150 (one based counting) becomes a location
 |  of [122:150] (zero based counting).
 |  
 |  >>> from Bio.SeqFeature import FeatureLocation
 |  >>> f = FeatureLocation(122, 150)
 |  >>> print(f)
 |  [122:150]
 |  >>> print(f.start)
 |  122
 |  >>> print(f.end)
 |  150

In [16]:
resistance_genes_coords[0].location.start


ExactPosition(90210)

In [20]:
gene_lol = list()
genome_end = 5131220
span = 100000
for gene in resistance_genes_coords:
    start = int(gene.location.start)
    end = int(gene.location.end)
    span_start = start - span
    # check if it's negative
    # if span_start < 0:
    #    span_start = genome_end + span_start
    span_end = end + span
    row = [gene.id, start, end, span_start, span_end, int(gene.location.strand)]
    gene_lol.append(row)

In [21]:
coordinates = pd.DataFrame(columns=["gene_id", "gene_start", "gene_end", "span_start", "span_end", "strand"], data=gene_lol)
coordinates

,gene_id,gene_start,gene_end,span_start,span_end,strand
0,HOKEGFAO_00108_gene,90210,90543,-9790,190543,-1
1,HOKEGFAO_00337_gene,354630,355797,254630,455797,-1
2,HOKEGFAO_00388_gene,414123,415371,314123,515371,1
3,HOKEGFAO_00389_gene,415370,418493,315370,518493,1
4,HOKEGFAO_00390_gene,418493,421571,318493,521571,1
5,HOKEGFAO_00392_gene,422983,424387,322983,524387,1
6,HOKEGFAO_00393_gene,424383,425106,324383,525106,1
7,HOKEGFAO_00522_gene,560973,562617,460973,662617,-1
8,HOKEGFAO_00549_gene,604627,605986,504627,705986,-1
9,HOKEGFAO_00563_gene,620511,621480,520511,721480,1


Now collect the spans from the genome
Use bed tools - create a bed file
something should be done with the negative coordinates
